# Exploring the Model Control Plane Backend: A Practical Guide
Welcome to our tour into the Model Control Plane, where we'll focus on two independent pipelines within ZenML. Each pipeline works on its own, creating specific artifacts. But what's fascinating is that these seemingly separate pipelines are intricately connected, all with the goal of delivering precise predictions.

Before the Model Control Plane, connecting these pipelines and consolidating everything was a challenge. Imagine extracting a trained model artifact from the training pipeline and smoothly integrating it into the predictions pipeline. Previously, this involved complex ID references, leading to constant config updates, or blindly relying on the latest training run. But what if that run didn't meet the necessary performance standards? Using a subpar model for predictions was out of the question, especially for vital applications!

Enter the Model Control Plane. This feature empowers you to effortlessly group pipelines, artifacts, and crucial business data into a unified entity: a `Model`. A Model captures lineage information and more. Within a Model, different `Model Versions` can be staged. For example, you can rely on your predictions at a specific stage, like `Production``, and decide whether the Model Version should be promoted based on your business rules during training. Plus, accessing data from other Models and their Versions is just as simple, enhancing the system's adaptability.

## Run on Colab

You can use Google Colab to run this notebook, no signup / installation required!



[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zenml-io//zenml-plugins/blob/main/model_control_plane/run.ipynb)

## Overview of the process

![Pipelines Overview](_assets/train_prediction_example.png)

Each time the `train_and_promote` pipeline runs, it creates a new iris_classifier. However, it only promotes the created model to `Production` if a certain accuracy threshold is met. The `do_predictions` pipeline simply picks up the latest Promoted model and runs batch inference on it. That way these two pipelines can independently be run, but can rely on each others output.

## Installation and Initialization


In [ ]:
from zenml.environment import Environment

if Environment.in_google_colab():
    # Install Cloudflare Tunnel binary
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudf

In [ ]:
!pip install "zenml[server,dev]>=0.45.5"

In [ ]:
# This might lead to a blocked cell, if its blocking just stop the notebook and skip this cell (server has been started)
!zenml up --help

### Warning: This will remove all your local ZenML artifacts. If you don't want this then just remove the zenml clean command

In [ ]:
!zenml clean -y  # remove this if you dont want to lose local data

In [ ]:
!zenml integration install sklearn -y

import IPython
from zenml import show
IPython.Application.instance().kernel.do_shutdown(restart=True)

In [ ]:
!rm -rf .zen
!zenml init

## Training pipeline: Create models and model versions

The Training pipeline orchestrates the training of a model object, storing datasets and the model object itself as links within a newly created Model Version. This integration is achieved by configuring the pipeline within a Model Context using `ModelConfig`. The name and `create_new_model_version` fields are specified, while other fields remain optional for this task.



In [ ]:
!zenml model list

In [ ]:
from zenml import get_step_context, step
from zenml.enums import ModelStages
from zenml.logger import get_logger

logger = get_logger(__name__)


@step
def promote_model(score: float):
    logger.info(f"The latest model score is: {score}")
    if score > 0.7:
        logger.info("Passed quality control... Promoting.")
        model_config = get_step_context().model_config
        model_version = model_config._get_model_version()
        model_version.set_stage(ModelStages.PRODUCTION, force=True)
    else:
        logger.info("Latest model failed quality control. Not promoting.")

In [ ]:
from zenml import pipeline
from zenml.model import ModelConfig

from steps.train.load import load_data
from steps.train.train import train_and_evaluate


@pipeline(
    enable_cache=False,
    model_config=ModelConfig(
        name="iris_classifier",
        license="Apache",
        description="Show case Model Control Plane.",
        create_new_model_version=True,
        delete_new_version_on_failure=True,
    ),
)
def train_and_promote_model():
    train_data, test_data = load_data()
    _, score = train_and_evaluate(train_data=train_data, test_data=test_data)
    promote_model(score=score)


train_and_promote_model()

### See results

Running the training pipeline creates a model and a Model Version, all while maintaining a connection to the artifacts.

Once it's done, check the results to see the newly created entities:



In [ ]:
# new model `iris_classifier` created
!zenml model list

In [ ]:
# new model version `1` created
!zenml model version list iris_classifier

In [ ]:
# list generic artifacts - train and test datasets are here
!zenml model version artifacts iris_classifier 1

In [ ]:
# list model objects - trained classifier here
!zenml model version model_objects iris_classifier 1

In [ ]:
# list deployments - none, as we didn't link any
!zenml model version deployments iris_classifier 1

In [ ]:
# list runs - training run linked
!zenml model version runs iris_classifier 1

### Run pipeline again and see new version

In [ ]:
train_and_promote_model()

In [ ]:
# new model version `2` created and promoted to production
!zenml model version list iris_classifier

See the results in the ZenML dashboard (This is a live session, you can come back to it anytime)

In [ ]:
show()

## Predictions pipeline: Use the latest production model to infer results

### Predictions pipeline

The Predictions Pipeline reads a trained model object from the Model Version labeled as Production. Here, the `version` is set to a specific stage, ensuring consistency across multiple runs. This approach shields the pipeline from the underlying complexities of the Training pipeline's promotion logic.

Given the frequent execution of the predictions pipeline compared to the training pipeline, we link predictions as versioned artifacts. The `overwrite` flag in the artifact configuration controls this, allowing for a comprehensive historical view.


Need to use a specific model version, not limited to stages? No problem. You can represent this either by version number or name, ensuring flexibility in your workflow.

In [ ]:
from zenml import get_pipeline_context, pipeline
from zenml.artifacts.external_artifact import ExternalArtifact
from zenml.enums import ModelStages
from zenml.model import ModelConfig

from steps.predict.load import load_data
from steps.predict.predict import predict


@pipeline(
    enable_cache=False,
    model_config=ModelConfig(
        name="iris_classifier",
        version=ModelStages.PRODUCTION,
    ),
    extra={"trained_classifier": "iris_classifier"},
)
def do_predictions():
    trained_classifier = get_pipeline_context().extra["trained_classifier"]
    inference_data = load_data()
    predict(
        model=ExternalArtifact(
            model_artifact_name=trained_classifier
        ),  # model_name and model_version derived from pipeline context
        data=inference_data,
    )


do_predictions()

### Artifacts Exchange Between Pipelines: Seamless Integration

In this pipeline, artifacts linked during the training stage are passed on. Leveraging `ExternalArtifact`, we effortlessly pass previously linked artifacts without repeating the model name and version setup.

*Handy Tip*: Explore further possibilities by using the `model_name` and `model_version` attributes of `ExternalArtifact` to pull artifacts from other models.

```python
from zenml.artifacts.external_artifact import ExternalArtifact

@pipeline(
    model_config=...,
    extra={"trained_classifier": "iris_classifier"},
)
def do_predictions():
    ...
    predict(
        model=ExternalArtifact(
            model_artifact_name=trained_classifier
        ),  # model_name and model_version derived from pipeline context
        ...
   

Executing the prediction pipeline ensures the use of the Model Version in Production stage, generating predictions as versioned artifacts.
 )
    ...
```

In [ ]:
from zenml import pipeline
from zenml.model import ModelConfig

from steps.train.load import load_data
from steps.train.promote import promote_model
from steps.train.train import train_and_evaluate


@pipeline(
    enable_cache=False,
    model_config=ModelConfig(
        name="iris_classifier",
        license="Apache",
        description="Show case Model Control Plane.",
        create_new_model_version=True,
        delete_new_version_on_failure=True,
    ),
)
def train_and_promote_model():
    train_data, test_data = load_data()
    _, score = train_and_evaluate(train_data=train_data, test_data=test_data)
    promote_model(score=score)

train_and_promote_model()

In [ ]:
# no new model version created, just consuming existing model
!zenml model version list iris_classifier

# list train, test and inference datasets and predictions artifacts
!zenml model version artifacts iris_classifier 1

Fantastic! By reusing the model version in the Production stage, you've connected the inference dataset and predictions seamlessly. All these elements coexist within the same model version, allowing effortless tracing back to training data and model metrics.

And what if you run the prediction pipeline again?


In [ ]:
do_predictions()

In [ ]:
# list train, test datasets and two version of 
# inference dataset and prediction artifacts
!zenml model version artifacts iris_classifier 1

# list runs, prediction runs are also here
!zenml model version runs iris_classifier 1

In [ ]:
from zenml import show

Everything worked seamlessly! You've added two more links to your artifacts, representing new predictions and inference dataset versions. Later, this detailed history can aid analysis or retrieving predictions from specific dates. Additionally, the prediction pipeline runs are conveniently attached to the same model version, ensuring you always know which code interacted with your models.

### More Command-Line Features

Explore additional CLI capabilities, like updating existing models and creating new ones, using straightforward commands.


In [ ]:
#### Updating Existing Models via CLI
!zenml model update iris_classifier -t tag1 -t tag2 -e "some ethical implications"

In [ ]:
#### Creating a Model via CLI
!zenml model register -n iris_classifier_cli -d "created from cli" -t cli

In [ ]:
### Well done! Time for a Quick Cleanup

!zenml model delete iris_classifier_cli
!zenml model delete iris_classifier -y

Nicely done, and now your workspace is tidy! Feel free to reach out if you have any more questions or if there's anything else you'd like to explore. Happy modeling! 😊